In [2]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime

In [2]:
datapath='Processed_Data/city_compare.h5'
db=pd.HDFStore(datapath)
db.keys()
# db.close()

['/airports', '/amap', '/hdl', '/heytea', '/hotel', '/movie', '/subway']

In [9]:
airports=db['airports'].drop(columns='date')
amap=db['amap'].drop(columns='date')
hdl=db['hdl'].drop(columns='date')
heytea=db['heytea'].drop(columns='date')
hotel=db['hotel'].drop(columns='date')
subway=db['subway'].drop(columns='date')
movie=db['movie'].drop(columns='date')

In [22]:
city_popu=pd.read_hdf('./Processed_Data/citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
city_popu

,城市,GDP,人均可支配收入,常住人口
0,合肥,10046,48283,512
1,阜阳,2805,34562,213
2,安庆,2468,35947,80
3,六安,1670,33647,197
4,滁州,3032,36051,78
...,...,...,...,...
292,丽水,1540,48532,56
293,嘉兴,5510,64124,152
294,舟山,1512,63702,88
295,绍兴,6001,66694,300


In [9]:
# city_income=pd.read_clipboard()
city_income=pd.read_hdf('./Processed_Data/citypopu.h5',key='cityincome')
city_income

,城市,人均可支配收入
0,上海,"76,437"
1,北京,"75,602"
2,苏州,"70,966"
3,杭州,"68,666"
4,广州,"68,304"
...,...,...
292,潮州,"26,440"
293,七台河,"26,315"
294,鸡西,"25,957"
295,鹤岗,"24,521"


In [17]:
# city_GDP=pd.read_clipboard()
city_GDP.to_hdf('./Processed_Data/citypopu.h5',key='raw2')
# city_GDP.merge(city_income,how='left',on='城市').merge(city_popu,how='left',on='城市')

In [20]:
df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
df_covid=df_covid[['city','peak','progress_2']]
df_covid.columns=['城市','峰值日','covid_progess']
df_covid['covid_progess']=pd.to_numeric(df_covid['covid_progess'])/100
df_covid.head(2)

,城市,峰值日,covid_progess
0,南昌,2022/12/20,0.0
1,武汉,2022/12/15,0.0


In [13]:
hdl[hdl['城市']=='莆田']

,城市,hdl
70,莆田,0.176434


In [21]:
all=city_income.merge(city_popu,how='left',on='城市').merge(df_covid,how='left',on='城市').merge(amap,how='left',on='城市').merge(airports,how='outer',on='城市').merge(subway,how='outer',on='城市').merge(heytea,how='outer',on='城市').merge(movie,how='outer',on='城市').merge(hdl,how='outer',on='城市').merge(hotel,how='outer',on='城市')


all['N']=all.count(axis=1)
all=all[all['covid_progess']>0]

all=all.sort_values(by='N',ascending=False).head(50).sort_values(by='常住人口',ascending=False).head(35).drop(columns='N')
all.to_clipboard(index=False)
